In [1]:
import os
from typing import Mapping, Optional, Sequence

from dotenv import load_dotenv
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list

# Source table
CATALOG = "isis"
SOURCE_DB = "cleaned"
SCHEDULE_TABLE = "running_schedule"

OPRALOG_LOGBOOK_ENTRY = "opralog_logbook_entry" 
OPRALOG_LOGBOOK_ENTRY_COMMENT = "opralog_logbook_entry_comment" 
OPRALOG_EDR_SOURCE = "MCR Running Log"

# Destination
TARGET_DB = "modeled"
EDR_TABLE = "mcr_edr"


load_dotenv()


True

In [2]:
spark = (
    SparkSession.builder
        .master("spark://data-accelerator.isis.cclrc.ac.uk:7077")
        .getOrCreate()
)
spark.active()

In [3]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{TARGET_DB}")
spark.sql(f"USE {CATALOG}.{TARGET_DB}")

""


In [5]:
# EDR table

edrtable_ensure_exists = f"""
CREATE TABLE IF NOT EXISTS {EDR_TABLE} (
  entry_id LONG,
  time_logged TIMESTAMP,
  cycle_name STRING,
  interval_label STRING,
  equipment STRING,
  downtime_mins DOUBLE,
  group STRING,
  comment_text STRING
)
USING iceberg
PARTITIONED BY (month(time_logged),equipment)
"""
spark.sql(edrtable_ensure_exists)

""


In [7]:
snapshot_name = "snapshot"
snapshot = f"""
CREATE OR REPLACE TEMPORARY VIEW {snapshot_name} AS (
    WITH downtimes AS (
      SELECT *
      FROM
        {SOURCE_DB}.{OPRALOG_LOGBOOK_ENTRY}
      WHERE logbook_name = '{OPRALOG_EDR_SOURCE}' AND column_title = 'Lost Time'
    ),
    groups AS (
      SELECT entry_id, string_data
      FROM
        {SOURCE_DB}.{OPRALOG_LOGBOOK_ENTRY}
      WHERE logbook_name = '{OPRALOG_EDR_SOURCE}' AND column_title = 'Group'
    ),
    equipment AS (
      SELECT entry_id, string_data
      FROM
        {SOURCE_DB}.{OPRALOG_LOGBOOK_ENTRY}
      WHERE logbook_name = '{OPRALOG_EDR_SOURCE}' AND column_title = 'Equipment'
    )
    SELECT
      a.entry_id,
      a.time_logged,
      e.cycle_name AS cycle_name,
      e.interval_label AS interval_label,
      c.string_data AS equipment,
      a.number_data AS downtime_mins,
      b.string_data AS group,
      d.comment_text
    FROM downtimes a
    INNER JOIN groups b
    INNER JOIN equipment c
    INNER JOIN {SOURCE_DB}.{OPRALOG_LOGBOOK_ENTRY_COMMENT} d
    INNER JOIN {SOURCE_DB}.{SCHEDULE_TABLE} e
    ON a.entry_id = b.entry_id AND b.entry_id = c.entry_id AND c.entry_id = d.entry_id AND a.time_logged BETWEEN e.started AND e.ended
)
"""
spark.sql(snapshot)

""


In [8]:
%%time

merge_snapshot = f"""
MERGE INTO {EDR_TABLE} t
USING {snapshot_name} s
ON t.entry_id = s.entry_id
WHEN NOT MATCHED THEN INSERT *
"""
spark.sql(merge_snapshot)

CPU times: user 39.3 ms, sys: 14.9 ms, total: 54.2 ms
Wall time: 3min 18s


""


In [9]:
%sql SELECT \
  time_logged, equipment, downtime_mins, comment_text \
FROM {EDR_TABLE} \
WHERE cycle_name = '2024/1' AND downtime_mins >= 180 \
ORDER BY downtime_mins DESC

time_logged,equipment,downtime_mins,comment_text
2024-05-14 14:02:06,Operational setup,9578.9,"Tripped on synchrotron beam loss monitors in SP7 and vacuum rise in same area. Machine physicist's working on trying to keep the beam on at low rep rate and try to diagnose what has happened and if they can make a difference with trim quad steering adjustments. SHIFT CHANGE @ 15:15 21:00 - Physicists have not identified the cause of the SP7 beamloss issues. We have decided to make entry to the Outer Synchrotron to check for physical damage and hotspots. Slightly raised radiation levels in SP7 but nothing high enough to cause a major concern. SHIFT CHANGE @ 23:15 Physicist's working through the night to try and diagnose the cause of the high beam loss and gradual vacuum increase under beam. SHIFT CHANGE @ 07:15 15/05/24 08:00 - Physicist's continuing to investigate the cause of the high beam losses in SP7. 10:00 - After more measurements from the machine physicists the decision was made to gain entry into the outer synchrotron to have a look in the beam aperture in SP7 doublets. Mechanical staff were contacted to carry out this work. Vacuum staff closed relevant valves and let up the area. RF staff ran down 1RF7 and 2RF6 and this was permitted off. As were trim quads and steerers in SP7 and SP6 by pulse power staff. Health physics carried out a survey in the area before any work was started. 12:00 - Using an endoscope, the mechanical staff located a damaged RF screen. The ceramic was broken and some of the wires had come loose and were sticking out into the beam. SHIFT CHANGE @ 15:15 15:50 - Looks like the doublet screen is shattered internally so cant be fixed in situe. Looks like some parts need machining. Will go and get a run down from A Hooper. 16:30 - After speaking to A Hooper and after J Ranner met with ILT, the decision has been made to replace the broken RF screen. 5 days off It looks to be too broken to repair in situ. Mechanical staff will remove the doublet from the machine tonight and leave it in the synchrotron. Work will begin tomorrow morning on a repair. This will entail removing the creen from the doublet, manufacturing a new screen by machining a stock screen so that it fits the doublet, installing new screen in the doublet then inserting the doublet back onto the machine. 17:00 - Inner and outer search lost due to north tunnel access being required. 17:45 - Ion Source group have taken the decision to turn the source off. C Talbott will come back in later. 20:00 - Ion Source switched off, doublet removed from machine and placed in the outer synch. 16/05/2024 - 18:00 Update from A Hooper. Broke screen is out of doublet and in a coffin by the end of the north tunnel. New screen ceramic machining is nearly finished. He will get all hands on assembly in the morning. Theres a small chance that if things go well tomorrow, they may be able to assemble the screen in the morning, get it into the synch, get it into the doublet and get it all back together by late tomorrow evening. This will mean that they can"
2024-05-12 10:59:59,Operational setup,4085.0,"Mains dip fault 13:15Hrs Contacted Estates help desk for information on power outage, they were unaware of a fault, but will investigate. The problem was tracked between Substation 14 & 15 on site, SSE were working on Harwell site and managed to reset the breakers but sufficient time off has caused the water systems to fail and therefore most of ISIS is off. There was a Solkor trip on the Harwell South Primary (HARS) E5L5 11kv CB while the SSEN works were being carried out on the other half of the board, this supply was restored within 10 minutes of the trip. The solkor should not have operated unless it saw an issue with the cable section between the substations. No work was being carried out in this room where the CB tripped and there was also no current being injected onto the other half of the board at the time of the event. Currently the reason for the trip is unknown.